## Summary

***Start port forwarding to the ELASPIC webserver machine:***

```bash
ssh -nNL 8001:localhost:8001 kimadmin@192.168.6.153
```

## Imports

In [ ]:
import os
import shlex
import subprocess
from datetime import datetime

import altair as alt
import pandas as pd
import sqlalchemy as sa
from dotenv import load_dotenv

In [ ]:
pd.set_option("max_rows", 1000)
pd.set_option("max_columns", 1000)

In [ ]:
load_dotenv("../.env.prod")

## Parameters

In [ ]:
import sys

sys.executable

## Load data

In [ ]:
DB_USER = os.environ["DB_USER"]
DB_PASSWORD = os.environ["DB_PASSWORD"]
DB_HOST = os.environ["DB_HOST"]
DB_PORT = os.environ["DB_PORT"]
DB_NAME = os.environ["DB_NAME"]

engine = sa.create_engine(f"mysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

#### `jobs_df`

In [ ]:
sql_query = """\
SELECT *
FROM `elaspic_webserver`.jobs
"""

jobs_df = pd.read_sql_query(sql_query, engine)

In [ ]:
jobs_df["status"] = jobs_df["isDone"].map({0: "not done", 1: "done"})

In [ ]:
display(jobs_df.head())

In [ ]:
len(jobs_df)  # 8325 10320

#### `job_to_mut_df`

In [ ]:
sql_query = """\
SELECT *
FROM `elaspic_webserver`.job_to_mut
"""

job_to_mut_df = pd.read_sql_query(sql_query, engine)

In [ ]:
display(job_to_mut_df.head())

#### `muts_df`

In [ ]:
sql_query = """\
SELECT *
FROM `elaspic_webserver`.muts
"""

muts_df = pd.read_sql_query(sql_query, engine)

In [ ]:
display(muts_df.head())

In [ ]:
len(muts_df)  # 108524 212734

## Make plots

In [ ]:
alt.data_transformers.disable_max_rows()

### All time

In [ ]:
alt.Chart(jobs_df).mark_bar().encode(
    #
    x="yearmonth(dateRun):T",
    y=alt.Y("count():T", title="Number of jobs"),
    color="status",
)

In [ ]:
alt.Chart(muts_df[muts_df["dateAdded"] >= jobs_df["dateRun"].min()]).mark_bar().encode(
    #
    x="yearmonth(dateAdded):T",
    y=alt.Y("count():T", title="Number of mutations"),
    color="status",
)

### Last year

In [ ]:
alt.Chart(jobs_df[jobs_df["dateRun"] >= datetime.fromisoformat("2019-05-01")]).mark_bar().encode(
    #
    x="yearmonth(dateRun):T",
    y=alt.Y("count():T", title="Number of jobs last year"),
    color="status",
)

In [ ]:
alt.Chart(muts_df[muts_df["dateAdded"] >= datetime.fromisoformat("2019-05-01")]).mark_bar().encode(
    #
    x="yearmonth(dateAdded):T",
    y=alt.Y("count():T", title="Number of mutations last year"),
    color="status",
)

## List failed jobs

In [ ]:
last_two_months_df = (
    muts_df[muts_df["dateAdded"] >= datetime.fromisoformat("2020-01-01")]
    .merge(job_to_mut_df, left_on="id", right_on="mut_id", how="left", suffixes=("_mut", "_job_to_mut"))
    .merge(jobs_df, left_on="job_id", right_on="jobID", suffixes=("_mut", "_job"))
)

len(last_two_months_df)  # 272296

In [ ]:
failed_with_emails_df = last_two_months_df[
    (last_two_months_df["status_mut"] != "done")
#     & last_two_months_df["email"].notnull()
#     & (last_two_months_df["email"] != "")
]#.drop_duplicates(subset=["jobID"])

len(failed_with_emails_df)  # 52098

In [ ]:
failed_with_emails_df.groupby('email')["id_mut"].count().sort_values(ascending=False)

In [ ]:
len(failed_with_emails_df)  # 52098

In [ ]:
# for key, gp in failed_with_emails_df.groupby(["inputIdentifier", "email"]):
#     print(key, ",".join(gp["protein"] + "." + gp["mut"]))